In [6]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


This notebook can produce a prediction file for the data of the test set for the six different methods that were required.
The cross validation of the hyperparameters is not covered in this file.

## Load the training data into feature matrix, class labels, and event ids:

In [7]:
from helpers import *
from implementations import *
from cleaning_data import *

In [8]:
DATA_TRAIN_PATH = 'train.csv'
DATA_TEST_PATH = 'test.csv' 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

##  Model:

To choose the model you will have to do the three essential following tasks: 

   1) Choose the type of regression: least squares or logistic regression.(To do so put a # in front of the unwanted choice and remove it in front of the wanted choice in the next cell)  
   
   2) Choose the parameters of the chosen method
   
   3) Choose the method. (To do so put a # in front of the unwanted choice and remove it in front of the wanted choice in the second next cell)

The run.py file and this notebook output with least_squares(y,x) and degree=8 are the same. Be sure to run the cells from the top to the bottom to produce the output file with the name OUTPUT_PATH.

In [22]:
#Choice of regression------------------------------------------------------------------------------------------

#choice='least squares'
choice='logistic regression'

#Choice of parameters------------------------------------------------------------------------------------------

degree=1                      #choice of the degree of the polynomial expansion
max_iters=50                  #max number of iterations for GD and SGD
gamma=0.007                    #step size for GD and SGD
lambda_=0.001                  #factor of the regularization for the ridge and logistic regression
OUTPUT_PATH='submission.csv'  

In [28]:
#Cleaning and adapting the data---------------------------------------------------------------------------------

x_0,x_1,x_23=adapt_x(tX,degree)
if choice=='least squares':
    y_0,y_1,y_23=adapt_y_least_squares(y,tX)
elif choice=='logistic regression':
    y_0,y_1,y_23=adapt_y_logistic(y,tX)
else:
    raise SyntaxWarning

#Computing the weights and loss on the training set-------------------------------------------------------------

#Gradient descent
#w_0, loss_w0 = least_squares_SGD(y_0, x_0, np.zeros(x_0.shape[1]), max_iters,gamma)
#w_1, loss_w1 = least_squares_SGD(y_1, x_1, np.zeros(x_1.shape[1]), max_iters,gamma)
#w_23, loss_w2 = least_squares_SGD(y_23, x_23, np.zeros(x_23.shape[1]), max_iters,gamma)

#Stochastic gradient descent
#w_0, loss_w0 = least_squares_SGD(y_0, x_0, np.zeros(x_0.shape[1]), max_iters,gamma)
#w_1, loss_w1 = least_squares_SGD(y_1, x_1, np.zeros(x_1.shape[1]), max_iters,gamma)
#w_23, loss_w2 = least_squares_SGD(y_23, x_23, np.zeros(x_23.shape[1]), max_iters,gamma)

#Least squares with normal equations
#w_0, loss_w0 = least_squares(y_0, x_0)
#w_1, loss_w1 = least_squares(y_1, x_1)
#w_23, loss_w2 = least_squares(y_23, x_23)

#Ridge regression
#w_0, loss_w0 = ridge_regression(y_0, x_0, lambda_)
#w_1, loss_w1 = ridge_regression(y_1, x_1, lambda_)
#w_23, loss_w2 = ridge_regression(y_23, x_23, lambda_)

#Logistic regression
#w_0, loss_w0 = logistic_regression(y_0, x_0, np.zeros(x_0.shape[1]), max_iters,gamma)
#w_1, loss_w1 = logistic_regression(y_1, x_1, np.zeros(x_1.shape[1]), max_iters,gamma)
#w_23, loss_w2 = logistic_regression(y_23, x_23, np.zeros(x_23.shape[1]), max_iters,gamma)

#Regularized logistic regression
w_0, loss_w0 = reg_logistic_regression(y_0, x_0, lambda_, np.zeros(x_0.shape[1]), max_iters,gamma)
w_1, loss_w1 = reg_logistic_regression(y_1, x_1, lambda_, np.zeros(x_1.shape[1]), max_iters,gamma)
w_23, loss_w2 = reg_logistic_regression(y_23, x_23, lambda_, np.zeros(x_23.shape[1]), max_iters,gamma)

#Predict on the train set----------------------------------------------------------------------------------------

y_pred_tr = label(w_0,w_1,w_23,x_0,x_1,x_23,tX,choice)
compute_accuracy(y, y_pred_tr)

0.672496

##  Create the submission file:

In [61]:
#Cleaning and adapting the data----------------------------------------------------------------------------------
x_0_te,x_1_te,x_23_te=adapt_x(tX_test,degree)

#Predict on the test set-----------------------------------------------------------------------------------------
y_pred_te=label_regression(w_0,w_1,w_23,x_0_te,x_1_te,x_23_te,tX_test,choice)
create_csv_submission(ids_test, y_pred_te, OUTPUT_PATH)